# Einstein Bianchi linearized equations


In [1]:
# neccessary imports
from ngsolve import *
from zenith import *
from ngsolve.webgui import Draw

importing NGSolve-6.2.2204


In [2]:
# create  a black hole at the origin
mass = 1
bh = BlackHole(mass=mass)

bh_list = [bh]
mesh = DefaultMesh(bh_list=bh_list, adaption=True, h=0.2, R=3)

{'bh_list': [<zenith.utils.CompactObjects.BlackHole object at 0x7f6b2d66e560>], 'adaption': True}


In [3]:
#Draw(mesh, clipping='z')

In [4]:

n = specialcf.normal(3)

def CurlTHcc2Hcd(E,dH):
    return InnerProduct(curl(E).trans, dH)*dx \
       +InnerProduct(Cross(E*n, n), dH*n)*dx(element_boundary= True)
def DivHcdHd(B,dv):
    return div(B)*dv*dx - B*n*n * dv*n * dx(element_boundary= True)

In [5]:
order = 2

fescc = HCurlCurl(mesh, order=order)
fescd = HCurlDiv(mesh, order=order)
fesd = HDiv(mesh, order=order, RT=True)

E, dE = fescc.TnT()
v, dv = fesd.TnT()
B, dB = fescd.TnT()

In [15]:
bfcurlT = BilinearForm(CurlTHcc2Hcd(E, dB))#, nonassemble = True)
bfcurl = BilinearForm(CurlTHcc2Hcd(dE, B))#, nonassemble = True)
bfdiv = BilinearForm(DivHcdHd(B, dv))#, nonassemble = True)
bfdivT = BilinearForm(DivHcdHd(dB, v))#, nonassemble = True)

bfcurl.Assemble()
bfcurlT.Assemble()
bfdiv.Assemble()
bfdivT.Assemble()

AttributeError: 'ngsolve.fem.CoefficientFunction' object has no attribute 'derivname'

In [7]:

with TaskManager():
    massE = BilinearForm(InnerProduct(E,dE)*dx, condense=True)
    preE = Preconditioner(massE, "bddc", block=True, blocktype="edgepatch")
    massE.Assemble()
    matE = massE.mat
    
    massEinvSchur = CGSolver (matE, preE)
    ext = IdentityMatrix()+massE.harmonic_extension
    extT = IdentityMatrix()+massE.harmonic_extension_trans
    massEinv =  ext @ massEinvSchur @ extT + massE.inner_solve

In [8]:
with TaskManager():
    
    massB = BilinearForm(InnerProduct(B,dB)*dx, condense=True)
    preB = Preconditioner(massB, "bddc", block=True, blocktype="edgepatch")
    massB.Assemble()
    matB = massB.mat    

    # preH = matH.CreateSmoother(fescd.FreeDofs(True), GS=False)

    massBinvSchur = CGSolver (matB, preB)
    ext = IdentityMatrix()+massB.harmonic_extension
    extT = IdentityMatrix()+massB.harmonic_extension_trans
    massBinv =  ext @ massBinvSchur @ extT + massB.inner_solve

In [9]:
with TaskManager():
    massv = BilinearForm(InnerProduct(v,dv)*dx, condense=True).Assemble()
    matv = massv.mat
    prev = matv.CreateSmoother(fesd.FreeDofs(True), GS=False)
    
    massvinvSchur = CGSolver (matv, prev)
    ext = IdentityMatrix()+massv.harmonic_extension
    extT = IdentityMatrix()+massv.harmonic_extension_trans
    massvinv =  ext @ massvinvSchur @ extT + massv.inner_solve
    

In [10]:
gfE = GridFunction(fescc)
gfB = GridFunction(fescd)
gfv = GridFunction(fesd)

In [11]:
E00 = 0
E01 = 16*sqrt(610)*z*(5*y**2 + 5*z**2 - 1)*exp(-10*x**2 - 10*y**2 - 10*z**2)/61
E02 = 16*sqrt(610)*y*(-5*y**2 - 5*z**2 + 1)*exp(-10*x**2 - 10*y**2 - 10*z**2)/61
E10 = 16*sqrt(610)*z*(5*y**2 + 5*z**2 - 1)*exp(-10*x**2 - 10*y**2 - 10*z**2)/61
E11 = -160*sqrt(610)*x*y*z*exp(-10*x**2 - 10*y**2 - 10*z**2)/61
E12 = 80*sqrt(610)*x*(y**2 - z**2)*exp(-10*x**2 - 10*y**2 - 10*z**2)/61
E20 = 16*sqrt(610)*y*(-5*y**2 - 5*z**2 + 1)*exp(-10*x**2 - 10*y**2 - 10*z**2)/61
E21 = 80*sqrt(610)*x*(y**2 - z**2)*exp(-10*x**2 - 10*y**2 - 10*z**2)/61
E22 = 160*sqrt(610)*x*y*z*exp(-10*x**2 - 10*y**2 - 10*z**2)/61
E = CoefficientFunction( (E00, E01, E02, E10, E11, E12, E20, E21, E22) , dims=(3,3) )

gfE.Set( E, bonus_intorder=10)

#Draw(gfE[0,1], mesh, 'E', clipping='z')

In [12]:
#peak = exp(-((x)**2+(y)**2+(z)**2)/ 2 )
#gfE.Set ( ((peak, 0,0), (0,0,0), (0,0,-peak) ))
#gfB.Set ( ((0,0,-peak), (0,0,0), (-peak,0,0) ))
#
#Draw(gfE[0,0], mesh, 'E', clipping='z')

In [13]:
t = 0
tend = 1
dt = 0.005
# tend = 5 * dt
#scene = Draw(Norm(gfB), mesh, clipping={"z":-1})
energyE = []
energyB = []
energytrace = []
energySym = []
energyv = []
scene = Draw(Norm(gfE), mesh, 'B', clipping='z')
with TaskManager(): # at each time step, save to a txt file the energy of the system
    while t < tend:

        
        hv = bfcurlT.mat * gfE.vec + bfdiv.mat.T * gfv.vec
        gfB.vec.data += dt * massBinv * hv
        gfv.vec.data += -dt * massvinv@bfdiv.mat * gfB.vec
        gfE.vec.data += -dt * massEinv@bfcurlT.mat.T * gfB.vec
        scene.Redraw()

        energyE.append (InnerProduct (gfE.vec, massE.mat * gfE.vec))
        energyB.append (InnerProduct (gfB.vec, massB.mat * gfB.vec))   
        energytrace.append (Integrate ( Norm (Trace(gfE)), mesh ))
        energySym.append (Integrate ( Norm (gfB-(gfB.trans)), mesh ))
        energyv.append (InnerProduct (gfv.vec, massv.mat * gfv.vec))
        t += dt

        # save the energy in a txt file
        with open("energy.txt", "a") as myfile: 
            myfile.write(str(t) + ";" + str(energyE[-1]) + ";" + str(energyB[-1]) + ";" + str(energytrace[-1]) + ";" + str(energySym[-1]) + ";" + str(energyv[-1]) + "\n")

    # print time in percentage and the firs 4 decimals of the energy 
        print ("t: ", int(t/tend*100), "%"+" E", round(energyE[-1],4), " B", round(energyB[-1],4), " trace", round(energytrace[-1],4), " sym", round(energySym[-1],4), " v", round(energyv[-1],4), end="\r")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

TypeError: matrix not ready - assemble bilinearform first

In [ ]:
import matplotlib.pyplot as plt

# plot the energy
plt.plot(energyE, label="E")
plt.plot(energyB, label="B")
plt.plot(energytrace, label="trace")
plt.plot(energySym, label="sym")
plt.plot(energyv, label="v")
# plot sum of energy E B v
plt.plot([energyE[i]+energyB[i] for i in range(len(energyE))], label="E+B+v")
plt.legend()
plt.show()


In [ ]:
# plot the value of the file energy.txt (the energy of the system) skip the first line
import matplotlib.pyplot as plt
import numpy as np
data = np.loadtxt("energy.txt", delimiter=";", skiprows=1)
plt.plot(data[:,0], data[:,1], label="E")
plt.plot(data[:,0], data[:,2], label="B")
plt.plot(data[:,0], data[:,3], label="trace")
plt.plot(data[:,0], data[:,4], label="sym")
plt.plot(data[:,0], data[:,5], label="v")
plt.legend()
# save the plot in a png file with high resolution
plt.savefig("energy.png", dpi=300)


In [ ]:
# save the plot in a png file
plt.savefig("energy.png")
plt.show()